In [0]:
USE SCHEMA projeto_datatran;

In [0]:
CREATE OR REPLACE VIEW workspace.projeto_datatran.vw_brasil_ano_ranking_letalidade AS
SELECT
  t.ano,
  count(*) AS total_acidentes,
  sum(f.total_vitimas) AS total_vitimas,
  sum(f.mortos) AS total_mortos,
  count_if(f.mortos > 0) AS acidentes_fatais,
  ROUND(count_if(f.mortos > 0) / NULLIF(COUNT(*), 0),2) AS taxa_letalidade
FROM workspace.projeto_datatran.gold_fact_accident f
JOIN workspace.projeto_datatran.gold_dim_time_sk t
  ON f.time_sk = t.time_sk
GROUP BY t.ano
ORDER BY t.ano


In [0]:
CREATE OR REPLACE VIEW workspace.projeto_datatran.vw_municipios_ano_ranking_letalidade AS
WITH municipios_top_20 AS (
  SELECT
    l.municipio
  FROM workspace.projeto_datatran.gold_fact_accident f
  JOIN workspace.projeto_datatran.gold_dim_location_sk l
    ON f.location_sk = l.location_sk
  GROUP BY l.municipio
  ORDER BY COUNT(*) DESC
  LIMIT 20
)

SELECT
  t.ano,
  l.municipio,
  COUNT(*)               AS total_acidentes,
  SUM(f.total_vitimas)   AS total_vitimas,
  SUM(f.mortos)          AS total_mortos,
  COUNT_IF(f.mortos > 0) AS acidentes_fatais,
  ROUND(
    COUNT_IF(f.mortos > 0) / NULLIF(COUNT(*), 0),
    2
  ) AS taxa_letalidade,
  RANK() OVER (
    PARTITION BY t.ano
    ORDER BY
      COUNT_IF(f.mortos > 0) / NULLIF(COUNT(*), 0) DESC
  ) AS ranking_letalidade
FROM workspace.projeto_datatran.gold_fact_accident f
JOIN workspace.projeto_datatran.gold_dim_time_sk t
  ON f.time_sk = t.time_sk
JOIN workspace.projeto_datatran.gold_dim_location_sk l
  ON f.location_sk = l.location_sk
JOIN municipios_top_20 m
  ON l.municipio = m.municipio
GROUP BY
  t.ano,
  l.municipio
ORDER BY
  t.ano,
  ranking_letalidade;